In [1]:
import os
import sys
from decimal import Decimal

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)


from data_sources.clob import CLOBDataSource
from research_notebooks.dneitor.utils import read_yaml_to_dict

clob = CLOBDataSource()
config = read_yaml_to_dict('configs/dneitor-binance_0.4.yml')

In [2]:
config

{'id': 'dneitor-binance_0.4',
 'controller_name': 'dneitor',
 'controller_type': 'generic',
 'manual_kill_switch': None,
 'candles_config': [],
 'markets': [{'connector_name': 'binance',
   'trading_pair': 'SOL-FDUSD',
   'total_amount_quote': 1156.51,
   'prices': [193.68,
    181.0926666666667,
    162.41727272727275,
    151.32538461538462,
    141.7084210526316,
    120.971],
   'amounts_quote_pct': [0.1, 0.1, 0.1, 0.1, 0.2, 0.2],
   'take_profit': 0.0026720986568548183,
   'activation_bounds': 0.002,
   'max_open_orders': 1},
  {'connector_name': 'binance',
   'trading_pair': 'PEPE-FDUSD',
   'total_amount_quote': 213.94,
   'prices': [1.24e-05,
    1.150173913043478e-05,
    9.785384615384616e-06,
    7.94090909090909e-06,
    6.986666666666666e-06,
    5.966666666666666e-06],
   'amounts_quote_pct': [0.1, 0.1, 0.1, 0.1, 0.2, 0.2],
   'take_profit': 0.0032328497204724845,
   'activation_bounds': 0.002,
   'max_open_orders': 1},
  {'connector_name': 'binance',
   'trading_pair': '

In [5]:
market_configs = config["markets"]
market_configs

[{'connector_name': 'binance',
  'trading_pair': 'SOL-FDUSD',
  'total_amount_quote': 1156.51,
  'prices': [193.68,
   181.0926666666667,
   162.41727272727275,
   151.32538461538462,
   141.7084210526316,
   120.971],
  'amounts_quote_pct': [0.1, 0.1, 0.1, 0.1, 0.2, 0.2],
  'take_profit': 0.0026720986568548183,
  'activation_bounds': 0.002,
  'max_open_orders': 1,
  'candles': <data_structures.candles.Candles at 0x152e602e0>},
 {'connector_name': 'binance',
  'trading_pair': 'PEPE-FDUSD',
  'total_amount_quote': 213.94,
  'prices': [1.24e-05,
   1.150173913043478e-05,
   9.785384615384616e-06,
   7.94090909090909e-06,
   6.986666666666666e-06,
   5.966666666666666e-06],
  'amounts_quote_pct': [0.1, 0.1, 0.1, 0.1, 0.2, 0.2],
  'take_profit': 0.0032328497204724845,
  'activation_bounds': 0.002,
  'max_open_orders': 1,
  'candles': <data_structures.candles.Candles at 0x152e607f0>},
 {'connector_name': 'binance',
  'trading_pair': 'WIF-FDUSD',
  'total_amount_quote': 207.46,
  'prices': [

In [11]:
import asyncio

# We are going to get the last 7 days of candles for each market and add it to the markets dictionary

tasks = [clob.get_candles_last_days(
    connector_name=market_config["connector_name"],
    trading_pair=market_config["trading_pair"],
    interval="15m",
    days=20,
) for market_config in market_configs]

candles = await asyncio.gather(*tasks)
for candle, market_config in zip(candles, market_configs):
    market_config["candles"] = candle

2024-08-13 22:57:43,632 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x15531b970>
2024-08-13 22:57:43,633 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1567ba1a0>, 1472.752709833)]']
connector: <aiohttp.connector.TCPConnector object at 0x15531abf0>
2024-08-13 22:57:43,659 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x156772f20>
2024-08-13 22:57:43,660 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1567b9420>, 1472.780013666)]']
connector: <aiohttp.connector.TCPConnector object at 0x156773070>
2024-08-13 22:57:43,669 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1552e0220>
2024-08-13 22:57:43,670 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1567b

In [12]:
market_config

{'connector_name': 'binance',
 'trading_pair': 'IO-FDUSD',
 'total_amount_quote': 201.85,
 'prices': [3.119,
  2.838785714285714,
  2.518666666666667,
  2.098666666666667,
  1.694375,
  1.4730714285714284],
 'amounts_quote_pct': [0.1, 0.1, 0.1, 0.1, 0.2, 0.2],
 'take_profit': 0.0029753161635884175,
 'activation_bounds': 0.002,
 'max_open_orders': 1,
 'candles': <data_structures.candles.Candles at 0x152f87ee0>}

In [13]:
# We are going to visualize one of the candles
candles_example = market_configs[0]["candles"]
candles_example.plot("candles")

In [14]:
candles_example.plot("returns")

In [15]:
for config in market_configs:
    fig = config["candles"].fig("candles")
    for price in config['prices']:
        fig.add_hline(y=price, line=dict(color='blue', dash='dash'))
    fig.show()